In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnull, lit, length, explode, count, upper, lower, regexp_replace, regexp_extract
from datapipelines.generate_clientes.commons.constants import *


In [ ]:
spark = SparkSession.builder \
    .appName("Data quality check enderecos_clientes ") \
    .config("spark.sql.parquet.enableVectorizedReader", "true") \
    .config("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED") \
    .getOrCreate()

In [7]:
df_ender_cli = spark.read.parquet("/app/data/processed/enderecos_clientes.parquet")
df_cli = spark.read.parquet("/app/data/processed/clientes.parquet")
df_opt = spark.read.parquet("/app/data/processed/clientes_opt.parquet")

In [10]:
df_cli.join(df_opt, on=CODIGO_CLIENTE, how="left_anti").show()
print("Clientes sem OPT:", df_cli.join(df_opt, on=CODIGO_CLIENTE, how="left_anti").count())


+--------------------+---------------+----+------------+-----+
|      codigo_cliente|data_nascimento|sexo|estado_civil|idade|
+--------------------+---------------+----+------------+-----+
|0000009DB36F622B7639|     2003-11-14|   F|         N/I|   22|
|000000F51C15031D708E|     2008-03-23|   F|         N/I|   17|
|00000F54BE2BBF0E7B13|     1986-02-24|   F|         N/I|   39|
|00003490C16D52FEB4C2|     2004-03-20|   F|         N/I|   21|
|00004A80EE1AE5324A9E|     1946-05-14| N/I|         N/I|   79|
|00004CAACD07633F2C95|     1978-11-26|   F|         N/I|   47|
|0000524920EA26B08218|     1999-09-14| N/I|         N/I|   26|
|0000566AFF0326105805|           NULL| N/I|         N/I| NULL|
|0000637A2669B1148663|     2003-01-24|   F|         N/I|   22|
|000069F01AA0A98858F4|     1943-07-10|   F|         N/I|   82|
|0000854EADF07158D702|     1959-09-05|   F|         N/I|   66|
|00008CC4B7481043B5B9|     1986-01-12|   F|           1|   39|
|0000A382CC46C42CB789|     2003-01-16| N/I|         N/I

In [11]:
df_cli.join(df_ender_cli, on=CODIGO_CLIENTE, how="left_anti").show()
print("Clientes sem ENDEREÇO:", df_cli.join(df_ender_cli, on=CODIGO_CLIENTE, how="left_anti").count())



+--------------------+---------------+----+------------+-----+
|      codigo_cliente|data_nascimento|sexo|estado_civil|idade|
+--------------------+---------------+----+------------+-----+
|00000F54BE2BBF0E7B13|     1986-02-24|   F|         N/I|   39|
|000013E1FB44D9A9E50F|     1984-12-13|   M|         N/I|   41|
|00003490C16D52FEB4C2|     2004-03-20|   F|         N/I|   21|
|000040FC122DB8AEEF51|     1982-04-06|   F|         N/I|   43|
|00004A80EE1AE5324A9E|     1946-05-14| N/I|         N/I|   79|
|00004CAACD07633F2C95|     1978-11-26|   F|         N/I|   47|
|00005651B2086081178C|     1961-06-19|   F|         N/I|   64|
|0000566AFF0326105805|           NULL| N/I|         N/I| NULL|
|0000573B0F66ED64C324|           NULL| N/I|         N/I| NULL|
|0000624AA6B5A0DF8CDB|     1992-09-03|   F|         N/I|   33|
|0000637A2669B1148663|     2003-01-24|   F|         N/I|   22|
|000069F01AA0A98858F4|     1943-07-10|   F|         N/I|   82|
|0000854EADF07158D702|     1959-09-05|   F|         N/I

In [13]:
df_full_joined = df_cli \
    .join(df_opt, on=CODIGO_CLIENTE, how="inner") \
    .join(df_ender_cli, on=CODIGO_CLIENTE, how="inner")

df_full_joined.show()
print("Total com correspondência nos 3 datasets:", df_full_joined.count())


+--------------------+---------------+----+------------+-----+--------------+-------------+---------------+--------------+---+-------------+
|      codigo_cliente|data_nascimento|sexo|estado_civil|idade|flag_lgpd_push|flag_lgpd_sms|flag_lgpd_email|flag_lgpd_call| uf|       cidade|
+--------------------+---------------+----+------------+-----+--------------+-------------+---------------+--------------+---+-------------+
|00001522AD94645C7688|     1992-11-10|   F|           2|   33|          true|         true|           true|          true| SC|     SAO JOSE|
|00001522AD94645C7688|     1992-11-10|   F|           2|   33|          true|         true|           true|          true| SC|     SAO JOSE|
|00001522AD94645C7688|     1992-11-10|   F|           2|   33|          true|         true|           true|          true| SC|FLORIANOPOLIS|
|0000161185110EE1BBE5|     1975-12-27|   F|         N/I|   50|         false|         true|          false|         false| SP|    SAO PAULO|
|0000269BBD18

In [ ]:
total_clientes = df_cli.select(CODIGO_CLIENTE).distinct().count()
clientes_com_opt = df_opt.select(CODIGO_CLIENTE).distinct().count()
clientes_com_endereco = df_ender_cli.select(CODIGO_CLIENTE).distinct().count()

print(f"Total de clientes             : {total_clientes}")
print(f"Clientes com dados de opt-in  : {clientes_com_opt}")
print(f"Clientes com endereço         : {clientes_com_endereco}")


Total de clientes             : 99970
Clientes com dados de opt-in  : 76623
Clientes com endereço         : 59181
